# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the TI-RUN package.
  * Set things up to carry out geometry optimization using Amber.

In [ ]:
(ql:quickload :ti-run)

In [ ]:
(leap:setup-amber-paths)

In [ ]:
(source "leaprc.protein.ff14SB")

# Start a new TI-RUN calculation

In [ ]:
(defparameter *ti-runs* (make-instance 'ti-run:calculation))

# Import the sketch from a Chemdraw file

In [ ]:
(png-from-file "ligand2.png" )

In [ ]:
(defparameter *sketch* (ti-run:load-chem-draw-ti-run "ligand2.cdxml"))

In [ ]:
(ti-run:setup-ligands *ti-runs* *sketch*)

In [ ]:
(defparameter v (show (ti-run:layout-ligands *ti-runs*)))
v

# Load the Thrombin PDB file

In [ ]:
(start-swank 4006)

In [ ]:
*default-pathname-defaults*

In [ ]:
(defparameter *thrombin* (load-pdb "2zff_fixed.pdb"))

In [ ]:
(build-unbuilt-hydrogens *thrombin*)

In [ ]:
(defparameter vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(nglv::add-representation vthrombin "ball+stick" :selection "all")

In [ ]:
(remove-molecules *thrombin* 'solvent)

In [ ]:
(cando:do-molecules (m *thrombin*) (format t "mol: ~a number-of-atoms -> ~a~%" m (chem:number-of-atoms m))
                    (cando:do-residues (res m) (format t "  res -> ~a~%" res)))

In [ ]:
(chem:setf-molecule-type (chem:content-with-name *thrombin* :|53U_3|) 'ligand)

In [ ]:
(push *thrombin* (ti-run:receptors *ti-runs*))

# Load a template ligand

In [ ]:
(defparameter *lig* (load-pdb "ligand.pdb"))

In [ ]:
(show *lig*)

In [ ]:
(defmacro named-test (name)
    `(cons ,name (lambda (a) (eq (chem:get-name a) ,name))))

Define a SMARTS matcher that labels ligand atoms

In [ ]:
(defparameter *tests* (mapcar (lambda (name) (named-test name)) '(:c13 :c12 :c15 :c9 :c8 :c7 :c5 :c3 :c1 :c20 :c18 :n1 :n3 :o2 :o1 :c2)))
(defparameter *pick* (chem:compile-smarts 
             "[C<c13>:13]1~[C<c12>:12]~C~C(~[C<c15>:15]~C1)~[C<c9>:9]~[N<n3>:33]~[C<c8>:8](~[O<o2>:22])[C<c7>:7]7C[C<c5>:5]CN7[C<c3>:3](~[O<o1>:21])[C<c2>:2]([N<n1>:31])[C<c1>:1]C10~C~[C<c20>:20]~C~[C<c18>:18]~C10" :tests *tests*))


Test the SMARTS code against the atoms of the ligand - it should recognize atom C13

In [ ]:
(cando:do-atoms (a *lig*) (when (chem:matches *pick* a)
                              (format t "Match!! ~a -> ~a~%" a (chem:matches *pick* a))))

# Use the GAFF force field to build chemically reasonable structures of candidate ligands

In [ ]:
(clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")
(load-amber-params "gaff.dat")

In [ ]:
(ti-run:build-ligands *ti-runs*)

In [ ]:
(show (ti-run:layout-ligands *ti-runs* :accessor 'ti-run::molecule))

# Pose the new ligands onto the template ligand

In [ ]:
(ti-run:pose-ligands-using-pattern *ti-runs* *pick* *lig* :stereochemical-restraints '( (2 . :R) (7 . :S)))

In [ ]:
(defparameter vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(defparameter *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (ti-run::molecule (first (ti-run:ligands *ti-runs*))))
(defparameter *struct* (make-instance 'cando-structure :matter *moveable-agg*))
(nglv::add-structure vthrombin *struct*)

In [ ]:
(ti-run:molecule (first (ti-run:ligands *ti-runs*)))

In [ ]:
(show (ti-run:layout-ligands *ti-runs* :accessor 'ti-run::molecule))

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(ti-run:build-job-nodes *ti-runs*)

In [ ]:
(ti-run::connect-job-nodes *ti-runs* :simple :connections 2 :stages 3 :windows 11)

In [ ]:
(ti-run:jobs *ti-runs*)

In [ ]:
(graph::save-graph *ti-runs*)

In [ ]:
*default-pathname-defaults*

In [ ]:
(defparameter *work-list* (ti-run::generate-jobs *ti-runs*))

In [ ]:
*work-list*